### 구글드라이브 연결
> - drive 폴더가 붙은 것을 확인한다.
> - drwx------ 4 root root 4.0K Mar 22 11:21 drive

In [1]:
# from google.colab import drive 
# drive.mount('/content/drive/')

### Install / Import 모듈 

In [2]:
# 코랩에 없는 라이브러리 설치해주기
# !pip install bayesian-optimization

In [3]:
import os
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
import matplotlib.pyplot as plt             # * 그래프 이미지

from tqdm import tqdm                       # 진행바

from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings           
import random                       
warnings.filterwarnings("ignore")           # 경고 문구 미표시

In [4]:
# 데이터가 많아 5분 정도 걸립니다.  
# 전체경로 = /content/drive/My Drive/Colab Notebooks/competition/c03_starcraft_prediction/data_raw/train.csv
# /content/drive/My Drive/Colab Notebooks/
# dir_base = '/content/drive/My Drive/Colab Notebooks/competition/''

dir_base = '/home/yk/0325_Starcraft/competition/'

raw      = 'c03_starcraft_prediction/data_raw/'
remake   = 'c03_starcraft_prediction/data_remake/'
submit   = 'c03_starcraft_prediction/data_submit/'

### 사용함수의 정의

In [5]:
os.chdir(dir_base)
from _assets.modules import *
from _assets.module_data_preps import *
from _assets.module_lgbm_model import *


# OS 화일 및 DF 정보조회를 위한 탐색 모듈


# GAME_ID 유니크 데이터 전처리를 위한 모듈 - baseline offer


# LGBM_CV 모델 - Dacon Baseline 참조



### Data 불러오기

| 컬럼명  | 값    | type     | 설명                  | 비고   |
|:-------:|:------|----------|-----------------------|--------|
| game_id |33781  |int       |경기 구분 기호         | unique (multiple) 
| winner  |0.6523 |float     |player-1 승리확률      | 0.6287
| time    |2.24   |float     |경기시간 (60분법 구분) | 2분24초
| player  |0      |int       |player 0 - 첫번째 선수 
|         |1      |int       |player 1 - 두번째 선수          
| species |T      |Object    |테란 (Terran)    |     
|         |P      |Object    |프로토스 (Protos)|      
|         |Z      |Object    |저그 (Zerg)      |      
| event   |Ability           |Object |생산, 공격 등 선수의 주요 행동
|         |AddToControlGroup |Object |부대에 추가
|         |Camera            |Object |시점 선택
|         |ControlGroup      |Object |부대 행동
|         |GetControlGroup   |Object |부대 불러오기
|         |Right             |Object |Click : 마우스 우클릭
|         |Selection         |Object |객체 선택
|         |SetControlGroup   |Object |부대 지정
|event_contents|@(좌표, 등)  | object|이벤트에 관한 상세설명 

In [6]:
# 화일위치 / list 확인하기 ... 사용함수
# show_ls('/content')          # drive 가 붙었는지 확인!
show_ls(dir_base + raw)      # 불러올 Data 화일이름 확인!

False
DIR_TARGET=/home/yk/0325_Starcraft/competition
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_raw/
----------------------------------------
  01. sample_submission.csv
  02. test.csv
  03. train.csv


In [7]:
%%time
# raw 데이터 3개 불러오기 ... 2분13초

train = pd.read_csv(dir_base + raw + "train.csv")                           # [ 67,091,776 x 7 ] ... 70%
test = pd.read_csv(dir_base + raw + "test.csv")                             # [ 28,714,849 x 6 ] ... 30%
sample_submission = pd.read_csv(dir_base + raw + "sample_submission.csv")   # [     16,787 x 2 ] ... 0.0175 %

CPU times: user 1min, sys: 4.69 s, total: 1min 4s
Wall time: 1min


In [19]:
show_infoDF_from(train)

*** DATA SHAPE = [ 67,091,776 x 7 ]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
game_id           int64
winner            int64
time              float64
player            int64
species           object
event             object
event_contents    object
dtypes: float64(1), int64(3), object(3)
memory usage: 3.5+ GB
None

            game_id        winner          time        player
count  6.709178e+07  6.709178e+07  6.709178e+07  6.709178e+07
mean   1.944981e+04  5.012076e-01  4.277912e+00  4.992699e-01
std    1.122980e+04  4.999985e-01  2.709741e+00  4.999995e-01
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
25%    9.718000e+03  0.000000e+00  2.120000e+00  0.000000e+00
50%    1.948500e+04  1.000000e+00  4.150000e+00  0.000000e+00
75%    2.919700e+04  1.000000e+00  6.360000e+00  1.000000e+00
max    3.887100e+04  1.000000e+00  1.059000e+01  1.000000e+00



### Data 전처리 및 저장하기
- 전처리 된 x_train, y_train 내용을 확인하고, 저장한다.

In [20]:
show_ls(dir_base + remake)

False
DIR_TARGET=/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_raw
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_remake/
----------------------------------------
  01. .ipynb_checkpoints
  02. accumujated_winning_rate_per_time.csv
  03. accumulated_winning_rate_per_time.csv
  04. df_new_rate04_correct.csv
  05. df_win_rate_every_10sec.csv
  06. df_win_rate_every_20sec.csv
  07. df_win_rate_every_40sec.csv
  08. df_xtest_remake.csv
  09. df_xtrain_remake.csv
  10. df_xtrain_remake_final.csv
  11. df_ytrain_remake.csv


In [36]:
%%time
"""
# 데이터 전처리 과정 - 약 7 ~ 10분 정도 걸립니다 ... 여기서 세션 리셋 됨!
# x_train, y_train = data_preparation(train, answer=True)           # [ 67,091,776 x 7 ] ... 70%
# show_infoDF_from(x_train) # *** DATA SHAPE = [ 38,872 x 27 ] ... [ game_id ... ]
# show_infoDF_from(y_train) # *** DATA SHAPE = [ 38,872 x 3 ] .. ['Unnamed: 0', 'game_id', 'winner']
"""

x_train = pd.read_csv(dir_base + remake + 'df_xtrain_remake.csv')  # to <class 'pandas.core.frame.DataFrame'>
ytrain = pd.read_csv(dir_base + remake + 'df_ytrain_remake.csv')  # to <class 'numpy.ndarray'>
y_train = ytrain['winner'].values                                 # array([1, 1, 0, ..., 0, 1, 0])

CPU times: user 52.3 ms, sys: 3.7 ms, total: 56 ms
Wall time: 55.4 ms


In [38]:
pd.Series(y_train).value_counts()
# 1    19499
# 0    19373            total = 38,872  y_train = np.array 38,872

1    19499
0    19373
dtype: int64

In [ ]:
# 제대로 작성이 됬는지 확인 다 끝났으면, 저장한다. 저장끝났으면 비활성화 처리


"""  # 저장했으면 비활성 처리
df_xtrain.to_csv(dir_base + remake + 'df_xtrain_remake.csv')
df_ytrain.to_csv(dir_base + remake + 'df_ytrain_remake.csv')
"""

# LGBM Classifier 모델 적용
 -  Light Gradient Boost Method 적용
 - [출처] :  YW & YY's Python, Machine & Deep Learning
 > - Light GBM: A Highly Efficient Gradient Boosting Decision Tree 논문 리뷰
 > - [Light GBM 모델 설명 및 사용법](https://greeksharifa.github.io/machine_learning/2019/12/09/Light-GBM/)

 > 다중 분류, 클릭 예측, 순위 학습 등에 주로 사용되는 Gradient Boosting Decision Tree (GBDT)는 굉장히 유용한 머신러닝 알고리즘이며, XGBoost나 pGBRT 등 효율적인 기법의 설계를 가능하게 하였다. 이러한 구현은 많은 엔지니어링 최적화를 이룩하였지만 고차원이고 큰 데이터 셋에서는 만족스러운 결과를 내지 못하는 경우도 있었다. 왜냐하면 모든 가능한 분할점에 대해 정보 획득을 평가하기 위해 데이터 개체 전부를 스캔해야 했기 때문이다. 이는 당연하게도, 굉장히 시간 소모적이다.


In [39]:
%%time
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score') 

# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'learning_rate': (0.0001, 0.1),    # learning_rate,    범위(0.0001~0.1)
        'num_leaves': (16, 1024),          # num_leaves,       범위(16~1024)
        'n_estimators': (16, 1024),        # n_estimators,     범위(16~1024)
        'subsample': (0, 1),               # subsample,        범위(0~1)
        'colsample_bytree': (0, 1),        # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),              # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),             # reg_lambda,       범위(0~50)
    }, 
    random_state=random.randrange(50000)   # 시드 고정 = 4321
)

# lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화 = Original Baseline!
lgbBO.maximize(init_points=20, n_iter=50)


"""
# LightGBM Classifier Document =  https://bit.ly/2QGqFTS
# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도 했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
"""

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.6204   |  0.7358   |  0.06047  |  1.018e+0 |  312.5    |  4.514    |  26.26    |  0.6482   |
|  2        |  0.6225   |  0.05301  |  0.05603  |  887.4    |  959.7    |  0.8974   |  42.0     |  0.5427   |
|  3        |  0.6181   |  0.7854   |  0.08622  |  1.017e+0 |  828.4    |  1.707    |  33.24    |  0.4024   |
|  4        |  0.6187   |  0.7415   |  0.07865  |  991.0    |  427.0    |  0.4041   |  48.8     |  0.01652  |
|  5        |  0.6234   |  0.3297   |  0.06287  |  943.6    |  677.2    |  5.565    |  30.78    |  0.7229   |
|  6        |  0.6193   |  0.1608   |  0.09457  |  977.3    |  714.1    |  0.8319   |  45.6     |  0.6512   |
|  7        |  0.6333   |  0.173    |  0.007023 |  79.3     |  544.6    |  6.916    |  29.8     |  0.2311   |
|  8      

|  45       |  0.6172   |  0.9907   |  0.09715  |  1.023e+0 |  1.021e+0 |  1.342    |  42.04    |  0.1331   |
|  46       |  0.6202   |  0.3974   |  0.0848   |  627.3    |  503.4    |  0.2638   |  3.66     |  0.9372   |
|  47       |  0.6392   |  0.9691   |  0.09016  |  154.7    |  108.4    |  0.5593   |  33.4     |  0.982    |
|  48       |  0.6315   |  0.7278   |  0.01679  |  913.4    |  518.2    |  3.636    |  1.677    |  0.08822  |
|  49       |  0.6266   |  0.5858   |  0.06378  |  546.8    |  269.1    |  2.364    |  49.06    |  0.5372   |
|  50       |  0.6295   |  0.4146   |  0.03048  |  554.3    |  694.2    |  1.803    |  1.803    |  0.7902   |
|  51       |  0.6429   |  0.8209   |  0.04652  |  590.1    |  17.09    |  0.4186   |  1.098    |  0.2813   |
|  52       |  0.6444   |  0.8022   |  0.0311   |  98.61    |  388.1    |  0.4149   |  46.01    |  0.763    |
|  53       |  0.6265   |  0.4797   |  0.06885  |  183.5    |  910.8    |  0.1184   |  1.955    |  0.9319   |
|  54     

### 3.0 모델 학습 및 검증
- Model Tuning & Evaluation

> 1. AUC가 가장 높은 하이퍼 파라미터를 사용해 최종 모델을 얻습니다.
> 1. 훈련 세트와 같은 방법으로 테스트 세트에서 Feature를 추출합니다.
> 1. 최종 모델을 사용해 예측을 수행합니다.
> 1. 예측 결과를 submission.csv로 저장합니다.

In [40]:
params = lgbBO.max['params']

models = lgb_cv(
        params['num_leaves'], 
        params['learning_rate'], 
        params['n_estimators'], 
        params['subsample'], 
        params['colsample_bytree'], 
        params['reg_alpha'], 
        params['reg_lambda'], 
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='model',
    )

# TEST를 풀기위해, 학습데이터로 변경 (data_preps)
 1. Train Data set = [ 67,091,776 x 7 ] ... 70%
  - game_id 별로 정리하면 [38,872 x 27]로 압축된다. 
  - 경기당, 평균 1,726개의 이벤트가 존재한다.(압축률 0.057939 %)
 > - x_train set = *** DATA SHAPE = [ 38,872 x 27 ] pandas.DataFrame
 > - y_train set = *** DATA SHAPE = [ 38,872 x 1 ]  np.array
 
 1. Test Data set  = [ 28,714,849 x 6 ] ... 30%
  - game_id 별로 정리하면 [16,787 x 27] 로 압축된다. 
  - 경기당 평균 1711개의 이벤트가 존재한다.(압축률 0.058461 %)
 > - x_test set = *** DATA SHAPE = [16,787 x 27] pandas.DataFrame 
 > - y_test set = *** DATA SHAPE = [16,787 x 1] np.array =  대회주관자가 가지고 있음
 
 1. Submission = y_test set 
  - submission = [16,787 x 1] np.array to [16,787 x 2 ] pandas.DataFrame --> df.csv

In [41]:
"""
# 테스트용 데이터를 전처리 해서 분석 준비 ... (시간소요) 미리 저장한 화일을 불러온다
# 저장했으면 비활성 처리
# x_test, _ = data_preparation(test, answer=False)                 # [ 28,714,849 x 6 ] ... 30%
# x_test.to_csv(dir_base + remake + 'df_xtest_remake.csv')
# x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')  # [  16,787 x 27 ]
"""
x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')    # *** DATA SHAPE = [ 16,787 x 27 ]


In [44]:
"""
# pd.read_csv('data/sample_submission.csv', index_col=0)
# sample_submission = pd.read_csv(dir_base + sub_base + "sample_submission.csv")
"""

preds = []

for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis=0)

In [45]:
show_ls(dir_base + raw)

False
DIR_TARGET=/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_remake
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_raw/
----------------------------------------
  01. sample_submission.csv
  02. test.csv
  03. train.csv


In [55]:
submission = pd.read_csv(dir_base + raw + "sample_submission.csv") 


submission['winner'] = pred
submission.to_csv(dir_base + submit + 'submission_way01_0412.csv', index=False)

# 결과 확인하기
- 3월 데이콘 대회 홈페이지 = https://bit.ly/39bqWVg
- 결과 제출하기 = https://dacon.io/competitions/official/235583/mysubmission/

In [48]:
df_1 = submission
df_1

,game_id,winner
0,38872,0.635284
1,38873,0.504019
2,38874,0.428862
3,38875,0.292274
4,38876,0.459122
...,...,...
16782,55654,0.642527
16783,55655,0.337637
16784,55656,0.781721
16785,55657,0.489813


In [52]:
%%time
"""
# df_time 데이터를 만든다. = 유니크 게임 갯수 *** DATA SHAPE = [ 38,872 x 1 ]
"""

df_time = pd.DataFrame(columns=['game_id'])       # 빈 df 선언
df_time['game_id'] = test['game_id'].unique() # 빈 df 에 'game_id 를 채운다
df_time['time'] = np.array(test[test.shift(-1).game_id != test.game_id].time)

CPU times: user 5.88 s, sys: 1.32 s, total: 7.2 s
Wall time: 1.94 s


In [54]:
df_time

,game_id,time
0,38872,7.06
1,38873,6.07
2,38874,4.41
3,38875,7.58
4,38876,2.50
...,...,...
16782,55654,6.14
16783,55655,9.58
16784,55656,6.24
16785,55657,1.17


In [ ]:
df_2 = pd.read_